In [ ]:
# Marketing EDA

Goal of notebook:
Register an embedding model and embed keywords
Explore the embeddings
# Cluster the embeddings f

### Run this one time

Guide from this blog

https://cloud.google.com/blog/products/data-analytics/introducing-bigquery-text-embeddings

In [1]:
# !pip install -q nomic
# !pip install -q datasets
# !pip install -q  transformers torch

In [2]:
# !bq mk --connection --location=us --project_id=cpg-cdp \
#     --connection_type=CLOUD_RESOURCE bq-vertex

# !bq show --connection 

# Here's how to do a LLM

In [4]:
# %%bigquery
# CREATE MODEL `cpg-cdp.genai_cap_v1.llm_model1`

# REMOTE WITH CONNECTION `cpg-cdp.us.bq-vertex`

# OPTIONS (remote_service_type = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1');

# Here's how to do embeddings

In [ ]:
# %%bigquery
# CREATE MODEL `cpg-cdp.genai_cap_v1.llm_embedding_model`
# REMOTE WITH CONNECTION `cpg-cdp.us.bq-vertex`
# OPTIONS(remote_service_type = 'CLOUD_AI_TEXT_EMBEDDING_MODEL_V1');

In [ ]:
%%bigquery 
create or replace table `cpg-cdp.trends_data.smartphones_13054_unitedstates_2840_embeddings_gecko` as (SELECT * FROM ML.GENERATE_TEXT_EMBEDDING(
MODEL `genai_cap_v1.llm_embedding_model`,
(SELECT term as content
FROM `cpg-cdp.trends_data.smartphones_13054_unitedstates_2840`
)));

Executing query with job ID: 3698be89-dcd3-444c-ad18-6124330fa313
Query executing: 162.62s

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL genai_cap_v1.clustering_smartphones
OPTIONS(model_type="KMEANS", num_clusters=10) 
AS (
SELECT text_embedding
FROM `cpg-cdp.trends_data.smartphones_13054_unitedstates_2840_embeddings_gecko`
);

In [6]:
import nomic
from nomic import atlas
import pandas as pd

In [13]:
token = ""

In [14]:
nomic.login(token)

In [41]:
%%bigquery unique_terms
SELECT distinct term as content
FROM `cpg-cdp.trends_data.smartphones_13054_unitedstates_2840`

Query is running:   0%|          |

Downloading:   0%|          |

In [42]:
total_unique_terms = len(unique_terms)
print("Total unique terms ", total_unique_terms)

Total unique terms  15270070


In [43]:
unique_terms_reset_idx = unique_terms # to get index field

In [47]:
data = {'index': [], 'content': []}
for i, content in enumerate(unique_terms_reset_idx.sample(10000)['content']):
    data['index'].append(i)
    data['content'].append(content)


In [51]:
df = pd.DataFrame(data)

In [53]:
project = atlas.map_text(data=df,
                         id_field='index',
                          indexed_field='content',
                          description='Smartphone Google Searches'
                          )

2023-09-01 20:37:33.185 | INFO     | nomic.project:_create_project:790 - Creating project `wistful-harmony` in organization `google`
2023-09-01 20:37:34.723 | INFO     | nomic.atlas:map_text:231 - Uploading text to Atlas.
2023-09-01 20:37:34.731 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:261 - id_field is not a string. Converting to string from int32

100%|██████████| 2/2 [00:01<00:00,  1.96it/s]
2023-09-01 20:37:35.759 | INFO     | nomic.project:_add_data:1422 - Upload succeeded.
2023-09-01 20:37:35.760 | INFO     | nomic.atlas:map_text:247 - Text upload succeeded.
2023-09-01 20:37:36.688 | INFO     | nomic.project:create_index:1132 - Created map `wistful-harmony` in project `wistful-harmony`: https://atlas.nomic.ai/map/8119fa22-1f6b-42d5-bb3e-65093b01edb3/14fe72b1-be37-4bba-8d58-0e1a68a6e3b7
2023-09-01 20:37:36.689 | INFO     | nomic.atlas:map_text:263 - wistful-harmony: https://atlas.nomic.ai/map/8119fa22-1f6b-42d5-bb3e-65093b01edb3/14fe72b1-be37-4bba-8d58-0e1a68a

In [56]:
project.maps[0]

wistful-harmony: https://atlas.nomic.ai/map/8119fa22-1f6b-42d5-bb3e-65093b01edb3/14fe72b1-be37-4bba-8d58-0e1a68a6e3b7